## Simpel tokenizers

Der findes en lang række eksisterende tokenizers fra forskellige pakker i Python. Disse kan med fordel bruges frem for at skrive sin egen tokenizer fra bunden.

SpaCy's sprogmodeller indeholder altid en tokenizer. Hvis man blot vil bruge selve tokenizeren uden de andre "processors" i modellen, kan man gøre det lig nedenstående:

In [1]:
import spacy # importer pakke

nlp = spacy.load('da_core_news_sm') # indlæs sprogmodel

tokenizer = nlp.tokenizer # hiv tokenizer ud af model (funktion for sig)

text = 'Politiet har givet borgerne råd' # tekststykke
tokens = list(tokenizer(text)) # brug tokenizer - her liste af tokens

print(tokens)

[Politiet, har, givet, borgerne, råd]


## Udvidet tokenizer

Man kan med fordel udnytte de forskellige måder, som spaCy's sprogmodeller behandler og analyserer tekst. 

I nedenstående defineres en udvidet tokenizerfunktion. Bemærk, hvordan de enkelte elementer af funktionen kan rettes til (hvilke stopord, hvilke part-of-speech tags, token længe, hvorvidt der skal bruges lemma eller ej).

In [3]:
def tokenizer(text):
    
    custom_stops = [""] # Definerer kontekstspecifikke stopord
    default_stopwords = list(nlp.Defaults.stop_words) # Indlæser prædefineret stopordsliste
    stop_words = default_stopwords + custom_stops # Danner samlet stopordsliste
    
    pos_tags = ['PROPN', 'ADJ', 'NOUN', 'VERB'] # Definerer POS-tags som skal bevares

    doc = nlp(text)

    tokens = []

    for word in doc: # Looper igennem hvert ord i tweet
        if (len(word.lemma_) < 3): # Ord må ikke være mindre end 3 karakterer - går videre til næste ord, hvis det er
            continue
        if (word.pos_ in pos_tags) and (word.lemma_ not in stop_words): # Tjek at ordets POS-tag indgår i listen af accepterede tags og at ordet ikke er stopord
            tokens.append(word.lemma_) # Tilføj ordets lemma til tokens, hvis if-betingelse er opfyldt
                
    return(tokens)

text = 'Politiet har givet borgerne råd' # tekststykke
tokens = tokenizer(text) # brug tokenizer - her liste af tokens

print(tokens)

['politi', 'give', 'borger', 'råd']


## Vectorizers

`sklearn` indeholder forskellige "text-vectorization"-funktioner; altså funktioner, der omdanner tekststykker til matematiske repræsentationer.

`CountVectorizer()` laver simple ordtællinger. Vectorizers fra `sklearn` bruges typisk på følgende måde:

1. Definér vectorizerfunktion (hvad skal funktionen, hvilke parametre)
2. Brug/"fit" funktionen på tekster/corpus (gerne som liste af strings)
3. Konvertér til passende datastruktur

In [5]:
texts = ['Vi er utroligt beærede',
         'Vi vil gerne dele prisen med alle',
         'Det skal vi have gjort op med.']

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer # importer CountVectorizer funktion

vectorizer = CountVectorizer() # dan vectorizerfunktion
transformed_documents = vectorizer.fit_transform(texts) # brug vectorizer på tekster

transformed_documents_as_array = transformed_documents.toarray()# Konverter fittet vectorizer til array

count_df = pd.DataFrame(transformed_documents_as_array, columns = vectorizer.get_feature_names_out()) # Konverter til data frame

count_df

,alle,beærede,dele,det,er,gerne,gjort,have,med,op,prisen,skal,utroligt,vi,vil
0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0
1,1,0,1,0,0,1,0,0,1,0,1,0,0,1,1
2,0,0,0,1,0,0,1,1,1,1,0,1,0,1,0


### TfidfVectorizer

`sklearn` har også en tf-idf vectorizer. Denne laver tf-idf vægtning af ord i teksten. 

*Bemærk:* Som standard normaliserer funktionen vægtene (kvadratsummen af vægte for hver tekst = 1 - se evt.: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer).

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer() # dan vectorizerfunktion
transformed_documents = vectorizer.fit_transform(texts) # brug vectorizer på tekster

# Konverter fittet vectorizer til array
transformed_documents_as_array = transformed_documents.toarray()

# Konverter til data frame
tfidf_df = pd.DataFrame(transformed_documents_as_array, columns = vectorizer.get_feature_names_out())

tfidf_df

,alle,beærede,dele,det,er,gerne,gjort,have,med,op,prisen,skal,utroligt,vi,vil
0,0.000000,0.546454,0.000000,0.000000,0.546454,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.546454,0.322745,0.000000
1,0.410747,0.000000,0.410747,0.000000,0.000000,0.410747,0.000000,0.000000,0.312384,0.000000,0.410747,0.000000,0.000000,0.242594,0.410747
2,0.000000,0.000000,0.000000,0.410747,0.000000,0.000000,0.410747,0.410747,0.312384,0.410747,0.000000,0.410747,0.000000,0.242594,0.000000


### Tilpasning af vectorizer - egen tokenizer

Vectorizer-funktionerne fra `sklearn` kan tilpasses på en lang række forskellige måder. En måde, som man kan tilpasse funktionerne på, er ved at bruge sin egen tokenizer. 

For at gøre dette, skal man blot have defineret en funktion, der omdanner et enkelt tekststykke til tokens (lig funktionen defineret tidligere i denne notebook). Denne sættes så ind som tokenizer, når man danner vectorizerfunktionen.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer = tokenizer) # dan vectorizerfunktion med egen tokenizer
transformed_documents = vectorizer.fit_transform(texts) # brug vectorizer på tekster

# Konverter fittet vectorizer til array
transformed_documents_as_array = transformed_documents.toarray()

# Konverter til data frame
tfidf_df = pd.DataFrame(transformed_documents_as_array, columns = vectorizer.get_feature_names_out())

tfidf_df

,beære,dele,pris
0,1.0,0.000000,0.000000
1,0.0,0.707107,0.707107
2,0.0,0.000000,0.000000


### Tilpasning af vectorizer - dokumentgrænser

En anden måde, som man kan tilpasse vectorizers, er ved at sætte minimum- og maksimumsgrænser for, hvor mange dokumenter, som ord skal indgå i.

Argumenterne `min_df` og `max_df` bruges til at sætte grænser for hhv. minimum antal dokumenter og maksimum antal dokumenter, som ord skal indgå i (df = "document frequency"). Ved begge kan man både angive absolutte tal eller andel.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 2) # dan vectorizerfunktion med egen tokenizer - ord skal indgå i to dokumenter
transformed_documents = vectorizer.fit_transform(texts) # brug vectorizer på tekster

# Konverter fittet vectorizer til array
transformed_documents_as_array = transformed_documents.toarray()

# Konverter til data frame
tfidf_df = pd.DataFrame(transformed_documents_as_array, columns = vectorizer.get_feature_names_out())

tfidf_df

,med,vi
0,0.000000,1.000000
1,0.789807,0.613356
2,0.789807,0.613356
